# Sample Evaluation and Cross Validation

Unit 2 / Lesson 3 / Assignment 3

It's come time to address another potential source of error in our models: __overfitting__.
__Overfitting__ is when your model is _so excessively complex that it starts to catch random noise_ instead of _describing the true underlying relationships_.

This is typically manifested with a model that _evaluates as more accurate than it really is_.
In most situations you shouldn't be able to build a perfect model, some error is to be expected.
__Overfitting__ is extremely common and easy to do, but there are ways to guard against it.
The main way is through how you evaluate your model.

Thus far we've been using our training data to evaluate our model.
By this we mean that we've used the same data to train the model and to see how well the model is doing.
When you think about it, some of the danger of that approach may become apparent.
If we create a very elaborate model it will pick up on the nuances of the data that are just from random noise.
If we evaluate the model on the training data, that ability to pick up noise will be returned as accuracy.
In reality, this isn't the case and doesn't depict how we'd really want to evaluate a model.
Generally we don't care about predicting things we already know.
We care about other data, new information, or other situations.
This is why testing with training data really isn't what we want to do. 

But if that's the case, what can we do?

### Holdout Groups

The simplest way to combat overfitting is with a **holdout group** (or sometimes "holdback group").
All this means is that you do not include all of your data in your training set, instead reserving some of it exclusively for testing.
While there is a cost to having less training data, your evaluation will be far more reliable.

When directly comparing two models that are based on different techniques or different specifications, this __holdout method__ combats __overfitting__.
Overfit models will see a drop in success rate outside of their training data, and so their performance will not be artificially inflated as it would be if you trained and validated your model using the whole data set.
This is because they got really good at matching the patterns within the data they were trained with, but didn't actually learn the things that matter but random noise.
When they try to match that random noise on new data their accuracy suffers.

How much data you choose to keep in a holdout is really up to you and depends on how much and what kind of data you have to begin with as well as what kind of model you're training.
You should check and see how much variance your model has as you add more data as well as how much data it would take to maintain a reasonably representative test sample.
It is, however, a balance.
30% is a common starting point, but really anything from 50% to 1% of the original dataset could be reasonable.

In [1]:
import pandas as pd
import sklearn
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Grab and process the raw data
PATH = ("https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/"
             "master/sms_spam_collection/SMSSpamCollection"
            )

sms_raw = pd.read_csv(PATH, delimiter= '\t', header=None)
sms_raw.columns = ['spam', 'message']

# Enumerate our spammy keywords.
keywords = ['click', 'offer', 'winner', 'buy', 'free', 'cash', 'urgent']

for key in keywords:
    sms_raw[str(key)] = sms_raw.message.str.contains(
        ' ' + str(key) + ' ',
        case=False
)

sms_raw['allcaps'] = sms_raw.message.str.isupper()
sms_raw['spam'] = (sms_raw['spam'] == 'spam')

sms_raw.head()

,spam,message,click,offer,winner,buy,free,cash,urgent,allcaps
0,False,"Go until jurong point, crazy.. Available only ...",False,False,False,False,False,False,False,False
1,False,Ok lar... Joking wif u oni...,False,False,False,False,False,False,False,False
2,True,Free entry in 2 a wkly comp to win FA Cup fina...,False,False,False,False,False,False,False,False
3,False,U dun say so early hor... U c already then say...,False,False,False,False,False,False,False,False
4,False,"Nah I don't think he goes to usf, he lives aro...",False,False,False,False,False,False,False,False


In [3]:
data = sms_raw[keywords + ['allcaps']]
target = sms_raw['spam']

from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
y_pred = bnb.fit(data, target).predict(data)

# display results
print('Number of mislabeled points out of a total {}: {}'.format(
    data.shape[0], (target != y_pred).sum()
))
print('Model accuracy:',
      ((data.shape[0] - (target != y_pred).sum()) / data.shape[0]*100),
      '%')

from sklearn.metrics import confusion_matrix
confusion_matrix(target, y_pred)

Number of mislabeled points out of a total 5572: 604
Model accuracy: 89.16008614501077 %


array([[4770,   55],
       [ 549,  198]], dtype=int64)

In [4]:
# Test your model with different holdout groups.

from sklearn.model_selection import train_test_split

# Use train_test_split to create the necessary training and test groups
X_train, X_test, Y_train, Y_test = train_test_split(data, target, test_size=0.2, random_state=20)

print('with 20% holdout: ' +
      str(bnb.fit(X_train, Y_train).score(X_test, Y_test)) )
print('testing on sample: ' + 
     str(bnb.fit(data, target).score(data,target)) )

with 20% holdout: 0.884304932735426
testing on sample: 0.8916008614501076


These scores look consistent, which means it _doesn't seem like our model is __overfitting___.
Part of the reason for that is that it's so simple (more on that in a bit).
We should look and see if any other issues are lurking here.
Let's try a more robust evaluation technique, __cross validation__.

### Cross Validation

__Cross validation__ is a more robust version of __holdout groups__.
Instead of creating just one holdout, you create several.

The way it works is this: start by breaking up your data into several equally sized pieces, or __folds__.
Let's say you make _x_ folds.
You then go through the training and testing process _x_ times, each time with a different fold held out from the training data and used as the test set.

The number of folds you create is up to you, but it will depend on how much data you want in your testing set.
At its most extreme, you're creating the same number of folds as you have observations in your data set.
This kind of cross validation has a special name: __Leave One Out__. Leave one out is useful if you're worried about single observations skewing your model, whereas large folds combat more general overfitting.

In [5]:
from sklearn.model_selection import cross_val_score

cross_val_score(bnb, data, target, cv=10)

array([0.89784946, 0.89426523, 0.89426523, 0.890681  , 0.89605735,
       0.89048474, 0.88150808, 0.89028777, 0.88489209, 0.89568345])

The array that `cross_val_score` returns is a series of accuracy scores with a different __hold out group__ each time.
If our model is __overfitting__ at a variable amount, those scores will fluctuate.
Instead, ours are relatively consistent.

Above we used the SKLearn built in functions for both of these kinds of cross validation, the documentation for which can be found [here](http://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators-with-stratification-based-on-class-labels).
However, the outputs from that are somewhat limited.
By default it uses the `score` method.
You can adjust what is returned, but you don't get all of the error types or outputs you may be interested in.
That's why it's not uncommon for people to code up their own cross validation.

In [6]:
# Implement your own cross validation with your spam model.



## What's a good score?

When we're looking at this model, we've been getting accuracy scores around 89%.
Intuitively that seems like a pretty good score, but in the start of this lesson we mentioned different kinds of error.
We also mentioned __class imbalance__.
Both of these things are at play here.
Using the topics we introduced earlier in this lesson, try to do a more in depth evaluation of the model looking at the kind of errors we're generating and what accuracy we'd get if we just randomly guessed.

You may want to use what's known as a [confusion matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html) to show different kinds of errors.

In [24]:
# Perform your additional evaluation here.

from sklearn.metrics import confusion_matrix
cf = confusion_matrix(target, y_pred)
print(cf)

[[4770   55]
 [ 549  198]]


In [23]:
print('{} false positives identified something as spam that is not'
      .format(cf[1,0]))
print('{} false negatives identified something as not spam when it is'
      .format(cf[0,1]))
print()

sen = (cf[1,1]/cf[1].sum()*100)
print('Sensitivity:', sen, '%')
print('{} percent of total {} positives correctly identified'
      .format(cf[1,1], cf[1].sum() ))
print()

spe = (cf[0,0]/cf[0].sum()*100)
print('Specificity:', spe, '%')
print('{} percent of total {} negatives correctly identified'
      .format(cf[0,0], cf[0].sum() ))

549 false positives identified something as spam that is not
55 false negatives identified something as not spam when it is

Sensitivity: 26.506024096385545 %
198 percent of total 747 positives correctly identified

Specificity: 98.860103626943 %
4770 percent of total 4825 negatives correctly identified


### Thinking like a Data Scientist

How you choose to validate your model in real life will depend upon the kind of data you're working with and the kinds of concerns you have about the model's performance.
Remember, your model is trained to fit the data you feed it, so if the situation changes your model will become less accurate.

For example, if there are seasonal changes to your observed variable but you only train on one month's data, you're going to have a problem.
You could test that by seeing how accurate your model is with a specific time period as your __holdout__, rather than a random sample.

### Overfitting and Naive Bayes

__Overfitting__ is always possible, but some models are more susceptible to it than others.
__Naive Bayes__ is actually pretty good at avoiding __overfitting__.
This is largely because the assumptions are so simple, particularly the assumed independence between any two independent variables.
One of the sources of __overfitting__ is when a model tries to map complex interactions between variables that aren't really there or significant.
__Naive Bayes__ cannot do this because it assumes they are all independent and therefore not interacting.
It's a nice characteristic at times, but it does mean it doesn't take into account how your features affect each other.

One final note on our models here.
They weren't __overfitting__, but they weren't telling us much either.
They were just barely more accurate than the dominant class.

Discuss with your mentor why that is and what you could do to improve the model.